In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

In [1]:
### Standard Imports
import numpy as np
import re
import sys
import os
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils
import lib.autoencoder_utilities as ae_utils

In [3]:
### Text Parameters
start_token = '<cls>'
end_token = '<eos>'
pad_token = '<pad>'
unk_token = '<unk>'
newline_token = '<new>'
mask_token = '<mask>'

### General Parameters
random_seed = 2022
model_folder = '../../../autoencoder/lstm/v5'
model_name = 'ae_lstm_att_mask_song'

### Model Parameters
val_split = 0.2
batch_size = 4
enc_dim, dec_dim = 256, 256
learn_rate = 0.001
epochs = 10
dropout = 0.05
recurrent_dropout = 0.05

In [4]:
os.makedirs(model_folder, exist_ok=True)

In [5]:
### Load Data
corpus = utils.load_corpus()
train_corpus, val_corpus, train_files, val_files = utils.split_corpus()

In [6]:
### Pre-Processing Text
_, word_count, index_to_vocab, vocab_to_index, songs, songs_token_ind = utils.tokenize_corpus(corpus,
                                                                            window_length = window_len,
                                                                            end_token = end_token,
                                                                            start_token = start_token,
                                                                            pad_token = pad_token,
                                                                            unk_token = unk_token,
                                                                            newline_token = newline_token,
                                                                            mask_token = mask_token,
                                                                                             padding = False)
vocab_size = len(word_count)

train_words, _, _, _, train_songs, train_songs_token_ind = utils.tokenize_corpus(train_corpus,
                                                                       window_length = window_len,
                                                                       index_to_vocab = index_to_vocab,
                                                                       vocab_to_index = vocab_to_index,
                                                                       end_token = end_token,
                                                                       start_token = start_token,
                                                                       pad_token = pad_token,
                                                                       unk_token = unk_token,
                                                                       newline_token = newline_token,
                                                                       mask_token = mask_token,
                                                                                padding = False)

val_words, _, _, _, val_songs, val_songs_token_ind = utils.tokenize_corpus(val_corpus,
                                                           window_length = window_len,
                                                           index_to_vocab = index_to_vocab,
                                                           vocab_to_index = vocab_to_index,
                                                           end_token = end_token,
                                                           start_token = start_token,
                                                           pad_token = pad_token,
                                                           unk_token = unk_token,
                                                           newline_token = newline_token,
                                                           mask_token = mask_token,
                                                                          padding = False)

In [18]:
max_length = max(len(song) for song in songs)
window_len = max_length

In [11]:
### Need to reorg - Encoder has whole song, decoder has all prev tokens, y is the next token

train_x_encoder, train_x_decoder, train_y = ae_utils.construct_song_seq(train_songs_token_ind,
                                                                        pad_index = vocab_to_index[pad_token],
                                                                        max_len = max_length,
                                                                        n_copies = 10,
                                                                        mask_prob = 0.15,
                                                                        mask_index = vocab_to_index[mask_token],
                                                                        random_seed = random_seed)

val_x_encoder, val_x_decoder, val_y = ae_utils.construct_song_seq(val_songs_token_ind,
                                                                        pad_index = vocab_to_index[pad_token],
                                                                        max_len = max_length,
                                                                        n_copies = 1,
                                                                        mask_prob = 0,
                                                                        mask_index = vocab_to_index[mask_token],
                                                                        random_seed = random_seed)

In [12]:
rand_int = np.random.randint(0, len(train_x_encoder), 1)[0]
print([index_to_vocab.get(x) for x in train_x_encoder[rand_int]])
print([index_to_vocab.get(x) for x in train_x_decoder[rand_int]])
print([index_to_vocab.get(y) for y in train_y[rand_int]])

rand_int = np.random.randint(0, len(val_x_encoder), 1)[0]
print([index_to_vocab.get(x) for x in val_x_encoder[rand_int]])
print([index_to_vocab.get(x) for x in val_x_decoder[rand_int]])
print([index_to_vocab.get(y) for y in train_y[rand_int]])

['<verse>', '<mask>', 'at', '<mask>', 'time', 'when', '<mask>', '<mask>', 'low', '<new>', 'the', 'journey', 'seemed', 'unsure', '<new>', 'but', 'through', 'it', 'all', '<new>', 'we', '<mask>', 'kept', 'the', 'flame', '<mask>', '<new>', '<mask>', '<mask>', '<new>', '<mask>', 'standing', '<mask>', 'and', '<mask>', '<new>', 'our', '<mask>', 'strong', 'and', 'free', '<new>', 'building', 'on', 'hopes', 'and', 'dreams', '<mask>', 'it', 'is', '<mask>', 'we', 'want', 'to', 'be', '<mask>', '<new>', '<chorus>', '<new>', 'let', '<mask>', 'reach', 'out', 'for', 'the', 'skies', '<new>', 'with', '<mask>', 'we', 'soar', '<mask>', 'high', '<new>', 'our', 'dreams', 'we', 'will', '<mask>', 'achieve', '<new>', 'we', '<mask>', 'make', 'our', 'destiny', '<new>', 'let', 'is', 'reach', '<mask>', 'for', 'the', 'skies', '<new>', 'with', 'wings', 'we', 'soar', 'up', 'high', '<new>', 'our', 'dreams', 'we', 'will', 'all', 'achieve', '<mask>', 'let', 'is', '<mask>', '<mask>', 'reach', 'for', '<mask>', 'skies', '<n

In [13]:
train_dataset = ae_utils.construct_datasets(train_x_encoder, train_x_decoder, train_y,
                                            random_seed = random_seed,
                                            batch_size = batch_size,
                                            vocab_size = vocab_size)
val_dataset = ae_utils.construct_datasets(val_x_encoder, val_x_decoder, val_y,
                                            random_seed = random_seed,
                                            batch_size = batch_size,
                                            vocab_size = vocab_size)

In [19]:
# Encoder
encoder_input = layers.Input(shape=(window_len,vocab_size), name = 'encoder_input')

# Return state in addition to output
encoder_output, encoder_hidden_state, encoder_cell_state = layers.LSTM(enc_dim,
                                                                       dropout = dropout, recurrent_dropout = recurrent_dropout,
                                                                       return_state=True, name = "encoder_lstm")(encoder_input)

# Decoder
decoder_input = layers.Input(shape=(window_len,vocab_size), name = 'decoder_input')

# Pass the encoder state to a new LSTM, as initial state
decoder_output = layers.LSTM(dec_dim,
                             dropout = dropout, recurrent_dropout = recurrent_dropout,
                             name="decoder_lstm")(decoder_input, initial_state=[encoder_hidden_state, encoder_cell_state])

# Attention
attention_context_vector = tf.keras.layers.Attention(name = 'attention')(inputs = [decoder_output, encoder_output])

# Output
output = layers.Dense(vocab_size, name = 'output', activation = 'softmax')(tf.concat([decoder_output, attention_context_vector], 1))

model = tf.keras.Model((encoder_input, decoder_input), output, name = model_name)
model.summary()

Model: "ae_lstm_att_mask_song"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 539, 1042)]  0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, 539, 1042)]  0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 256),        1330176     ['encoder_input[0][0]']          
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                              

In [20]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate),
              metrics = ['accuracy'])

In [21]:
### Callbacks
callback_es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=10,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

callback_mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_folder+'/weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [22]:
history = model.fit(x = train_dataset, validation_data = val_dataset, epochs = epochs, callbacks = [callback_es, callback_mc])

Epoch 1/10


ValueError: in user code:

    File "C:\Users\quekh\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\quekh\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\quekh\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\quekh\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\quekh\anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\quekh\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:

In [15]:
model.save_weights(f'{model_folder}/final_weights.hdf5')

In [16]:
#model.load_weights(f'{model_folder}/final_weights.hdf5')

In [17]:
prompts = ['Whenever I think back', 'And so this I know',
           'I am tired of being what you want me to be', 'Feeling so faithless, lost under the surface',
           'Relight our fire, we will find our way', 'We will rise stronger together']
result_strings = {}
results = {}
for prompt in prompts:
    result_str, result = utils.generate_text(model,
                                             ae_utils.ind_to_input_fun, ae_utils.update_input_fun,
                                             start_string = prompt,
                                             window_length = window_len,
                                             vocab_to_index_dict = vocab_to_index, index_to_vocab_dict = index_to_vocab,
                                             vocab_size = vocab_size,
                                             num_generate = 100, temperature = 1.0,
                                             random_seed = random_seed,
                                             end_token = end_token, start_token = start_token,
                                             pad_token = pad_token, unk_token = unk_token,
                                             newline_token = newline_token,
                                             depth = vocab_size,
                                             to_mask = True,
                                             mask_index = vocab_to_index[mask_token])
    result_strings[prompt] = result_str
    results[prompt] = result

In [18]:
print(result_strings)

{'Whenever I think back': 'Whenever I think back \n a \n unfurled \n <verse> \n <chorus> my said struggled singapore my we <chorus> do oh \n \n not our \n the stand the in \n till \n one out \n to \n now her \n sing \n where be are singapore \n first is her to be here and \n make i friends oh heart \n it <chorus> \n a oh together singapore oh must \n \n what favourite oh oh oh up oh oh oh awaits oh heart oh oh singaporean dating oh \n oh will oh different oh oh oh oh together oh oh oh \n', 'And so this I know': 'And so this I know \n much see \n \n we our as together \n then \n of you \n i \n together story we \n \n strong our higher \n your a who grown home a out far \n together out my the be tell were one worlds \n can set one you we our <verse> place sing whole are it wildest ever asking remembered \n my \n hand ourselves our', 'I am tired of being what you want me to be': 'I am tired of being what you want me to be light the \n to once \n they \n \n home heart love oh the \n \n the

In [19]:
for k, v in result_strings.items():
    with open(model_folder+f'/human_{model_name}-{utils.remove_punct(k.lower())}.txt', 'w') as f:
        f.write(v)